## Predicting level values from last 300 sec data using tflearn

In [1]:
from __future__ import division, print_function, absolute_import
import re, collections
import numpy as np
import pandas as pd

import tensorflow as tf
import tflearn
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d, conv_2d_transpose
from tflearn.layers.estimator import regression

from IPython import display
import matplotlib.pyplot as plt
% matplotlib inline
pd.set_option('max_rows', 100)
# np.set_printoptions(threshold=np.nan)

### Creating the new csv file with missing
Don't run the below cell as it takes a long time to generate new dataframe
The below lines read our csv and adds missing time second data to df2 as described in 

df = pd.read_csv('irage_ds_trim.csv',parse_dates = ['date'], 
                   infer_datetime_format = True)

df.insert(1,'time_diff',0)
df['time_diff'] = (df['date']-df['date'].shift()).fillna(0).apply(lambda x: x  / np.timedelta64(1,'s')).astype('int64') % (24*60*60)
df.head(n=50)

df2 = pd.DataFrame(columns = df.columns)
rows = len(df.index)
c=0
for i in range(rows-1):
    time_diff = df.iloc[i+1,1] 
    date_diff = df.iloc[i+1,0].day - df.iloc[i,0].day
    if time_diff >1 and date_diff==0:
        for t in range(0,time_diff):
            new_time = df.iloc[i,0] + pd.DateOffset(seconds=t)
            df2 = pd.concat([df2, df.iloc[[i]]], ignore_index=True)
            df2.iloc[len(df2.index)-1,0] = new_time
    else:
        df2 = pd.concat([df2, df.iloc[[i]]], ignore_index=True)
      
    
    if c==1000:
        display.clear_output(wait=True)
        print('At row: %s and %0.2f completed'%(i,i/rows*100))
        c=0
    c+=1


df2.drop(['time_diff'])
df2.to_csv('modified.csv', index = False)

In [3]:
df = pd.read_csv('modified.csv',parse_dates = ['date'], 
                 infer_datetime_format = True)

df.drop(['date','b0','a0','fut_direction'], 1, inplace = True)
df.sort_index(axis=1, inplace = True)
df

,bidx0,bidx1,bidx2,bidx3,bidx4,bsz0,bsz1,bsz2,bsz3,bsz4,sidx0,sidx1,sidx2,sidx3,sidx4,ssz0,ssz1,ssz2,ssz3,ssz4
0,35,25,60,35,65,1,2,1,1,1,35,15,50,20,70,2,1,1,2,2
1,30,25,55,35,60,1,1,1,1,1,30,5,45,20,50,1,2,1,1,2
2,30,20,50,35,55,1,1,1,1,1,30,5,45,20,50,2,2,1,1,2
3,30,10,60,20,65,1,1,1,1,1,30,5,45,20,50,1,1,1,1,2
4,25,20,45,30,55,1,1,1,1,1,25,5,30,20,45,2,1,1,1,1
5,25,20,45,30,55,1,1,1,1,1,25,5,30,20,45,2,1,1,1,1
6,25,20,45,30,55,1,1,1,1,1,25,5,30,20,45,2,1,1,1,1
7,25,10,55,20,65,1,1,1,1,2,25,5,30,20,45,2,2,1,1,1
8,25,10,55,20,65,1,1,1,1,2,25,5,30,20,45,2,2,1,1,1
9,25,10,55,20,65,1,1,1,1,2,25,5,30,20,45,2,2,1,1,1


In [4]:
# Bringing the data in the shape as shown in 'How data looks.xlsx excel file'
X = df.values

print('Original X')
print(X.shape)

seq_length = 300
img_height = 20
num_images = X.shape[0]//seq_length

X = X[:seq_length*num_images,:]
print('\nX after calculating num_images')
print(X.shape)

X = X.reshape(num_images,seq_length, img_height).astype("float32").transpose((0,2,1))

Y = np.array([X[i+1,:,0] for i,x in enumerate(X) if i<X.shape[0]-1]).astype("int64")
print('\nChecking Y \nFirst time series value of X[1]')
print(X[1,:,0])
print('Value of Y corresponding to X[0]')
print(Y[0])
print('\nX and Y after converting to 20x300 time-series images')
X = X[:-1,:,:]
print(X.shape)
print(Y.shape)

train_split = 0.85
split_val = round(num_images*train_split)

X_train = X[:split_val,:,:,np.newaxis]
X_test = X[split_val:,:,:,np.newaxis]
Y_train = Y[:split_val,:]
Y_test = Y[split_val:,:]

X_train -= np.mean(X_train, axis=0)
X_train /= np.std(X_train, axis=0)

X_test -= np.mean(X_train, axis=0)
X_test /= np.std(X_train, axis=0)

print('\nX and Y after train-test split, normalisation and channel dimension insertion')
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

Original X
(725175, 20)

X after calculating num_images
(725100, 20)

Checking Y 
First time series value of X[1]
[ 35.   5.  40.  15.  45.   1.   1.   1.   1.   1.  35.  20.  50.  25.  70.
   1.   1.   1.   1.   1.]
Value of Y corresponding to X[0]
[35  5 40 15 45  1  1  1  1  1 35 20 50 25 70  1  1  1  1  1]

X and Y after converting to 20x300 time-series images
(2416, 20, 300)
(2416, 20)

X and Y after train-test split, normalisation and channel dimension insertion
(2054, 20, 300, 1)
(362, 20, 300, 1)
(2054, 20)
(362, 20)


### Making the neural network

In [6]:
tf.reset_default_graph()
filter_size = 3
stride_y = 5
epochs = 50

# Convolutional network with 2 conv and 2 fully connected layers
network = input_data(shape=[None, 20, 300, 1])
conv1 = conv_2d(network, 32, filter_size, activation='relu', padding = 'same', strides= [1,1,stride_y,1], name = 'conv1')
pool1 = max_pool_2d(conv1, 2)
conv2 = conv_2d(pool1, 64, filter_size, activation='relu', padding = 'same', name ='conv2')
pool2 = max_pool_2d(conv2, 2)
fc1 = fully_connected(pool2, 512, activation='relu',name ='fc1')
drop1 = dropout(fc1, 0.5)
fc2 = fully_connected(drop1, 20, name ='fc2')
network = regression(fc2, optimizer='adam', loss='mean_square',
                     learning_rate=0.001, metric='R2')

# Training
model = tflearn.DNN(network, tensorboard_verbose=0)
model.fit(X_train, Y_train, n_epoch=epochs, shuffle=False, validation_set=(X_test, Y_test),
          show_metric=True, batch_size=26,run_id='irage')

conv1 = model.get_weights(conv1.W)
conv2 = model.get_weights(conv2.W)
fc1 = model.get_weights(fc1.W)
fc2 = model.get_weights(fc2.W)

Training Step: 3949  | total loss: 12.39048 | time: 2.359s
| Adam | epoch: 050 | loss: 12.39048 - R2: 0.9992 -- iter: 2028/2054
Training Step: 3950  | total loss: 12.57872 | time: 3.388s
| Adam | epoch: 050 | loss: 12.57872 - R2: 0.9999 | val_loss: 29608.31656 - val_acc: 81.1728 -- iter: 2054/2054
--


### Tried different parameters

filter_size = 3 stride y = 5 epoch = 50
Training Step: 3949  | total loss: 14.60604 | time: 2.270s
| Adam | epoch: 050 | loss: 14.60604 - R2: 1.0658 -- iter: 2028/2054
Training Step: 3950  | total loss: 14.87758 | time: 3.300s
| Adam | epoch: 050 | loss: 14.87758 - R2: 1.0415 | val_loss: 18890.15718 - val_acc: 54.9775 -- iter: 2054/2054

filter_size = 5 stride y = 5 epoch = 50
Training Step: 3949  | total loss: 16.56158 | time: 2.267s
| Adam | epoch: 050 | loss: 16.56158 - R2: 1.0335 -- iter: 2028/2054
Training Step: 3950  | total loss: 17.20631 | time: 3.305s
| Adam | epoch: 050 | loss: 17.20631 - R2: 1.0086 | val_loss: 23708.56095 - val_acc: 67.8062 -- iter: 2054/2054

filter_size = 5 stride y = 5 epoch = 30
Training Step: 2369  | total loss: 21.36651 | time: 2.831s
| Adam | epoch: 030 | loss: 21.36651 - R2: 0.9475 -- iter: 2028/2054
Training Step: 2370  | total loss: 20.76781 | time: 3.872s
| Adam | epoch: 030 | loss: 20.76781 - R2: 0.9431 | val_loss: 62821.49411 - val_acc: 163.0235 -- iter: 2054/2054

filter_size = 5 stride y = 5 epoch = 20
Training Step: 1579  | total loss: 21.93146 | time: 2.857s
| Adam | epoch: 020 | loss: 21.93146 - R2: 0.9790 -- iter: 2028/2054
Training Step: 1580  | total loss: 21.09883 | time: 3.910s
| Adam | epoch: 020 | loss: 21.09883 - R2: 0.9611 | val_loss: 60197.73860 - val_acc: 155.9296 -- iter: 2054/2054

filter_size = 5 stride y = 5 epoch = 10
Training Step: 789  | total loss: 27.39871 | time: 2.850s
| Adam | epoch: 010 | loss: 27.39871 - R2: 0.9602 -- iter: 2028/2054
Training Step: 790  | total loss: 29.12849 | time: 3.912s
| Adam | epoch: 010 | loss: 29.12849 - R2: 0.9374 | val_loss: 71309.16242 - val_acc: 183.2223 -- iter: 2054/2054

In [7]:
tf.trainable_variables()

[<tf.Variable 'conv1/W:0' shape=(3, 3, 1, 32) dtype=float32_ref>,
 <tf.Variable 'conv1/b:0' shape=(32,) dtype=float32_ref>,
 <tf.Variable 'conv2/W:0' shape=(3, 3, 32, 64) dtype=float32_ref>,
 <tf.Variable 'conv2/b:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'fc1/W:0' shape=(4800, 512) dtype=float32_ref>,
 <tf.Variable 'fc1/b:0' shape=(512,) dtype=float32_ref>,
 <tf.Variable 'fc2/W:0' shape=(512, 20) dtype=float32_ref>,
 <tf.Variable 'fc2/b:0' shape=(20,) dtype=float32_ref>]

### Tried saving array as image but coudn't

In [190]:
from PIL import Image

w, h = 512, 512
data = np.zeros((h, w, 3), dtype=np.uint8)
data[256, 256] = [255, 0, 0]
img = Image.fromarray(data, 'RGB')
img.save('my.png')

### Tried to do VisualBackProp 

In [ ]:
t = np.mean(conv2, axis=3, keepdims = True)
t = np.mean(t, axis=2, keepdims = True)

conv2_avg = tf.constant(t)
conv2_decon = conv_2d_transpose(conv2_avg, 1, filter_size, output_shape = [2, 2, 1])
conv1_avg = tf.constant(np.mean(conv1, axis=3))
conv1_conv2 = tf.multiply(conv1_avg,conv2_decon)
mask = conv_2d_transpose(conv1_conv2, 1, filter_size, [20,300,1])

sess = tf.Session()
init = tf.global_variables_initializer()
result = sess.run(mask)
print(result)